In [32]:
import pandas as pd
import numpy as np
import networkx as nx

#to calculate distance
def calculate_distance(x1, y1, z1, x2, y2, z2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

# Data
pipe_connections_data = pd.read_csv('/content/pipe_connections.csv')
pipe_nodes_data = pd.read_csv('/content/pipe_nodes.csv')
devices_data = pd.read_csv('/content/project_locations_users.csv')

# Add unique device identifiers
devices_data["device"] = devices_data["household"] + "_" + devices_data["name"]

# Boiler coordinates
boiler_coordinates = pipe_nodes_data.loc[pipe_nodes_data['pipe_ID'] == 'boiler', ['x', 'y', 'z']].values.flatten()

# Map each device to its nearest pipe node
def map_device_to_node(device, pipe_nodes):
    device_coordinates = device[['x', 'y', 'z']].values
    pipe_nodes['distance'] = pipe_nodes.apply(
        lambda node: calculate_distance(*device_coordinates, node['x'], node['y'], node['z']), axis=1
    )
    nearest_node = pipe_nodes.loc[pipe_nodes['distance'].idxmin()]
    return nearest_node['pipe_ID']

devices_data['nearest_pipe_node'] = devices_data.apply(
    lambda device: map_device_to_node(device, pipe_nodes_data), axis=1
)

# Calculate distances and energy losses for each device
avg_energy_loss = pipe_connections_data['loss_of_energy'].mean()

for index, device in devices_data.iterrows():
    distance = calculate_distance(*boiler_coordinates, device['x'], device['y'], device['z'])
    energy_loss_in_pipes = distance * avg_energy_loss

    devices_data.at[index, 'distance_to_boiler'] = distance
    devices_data.at[index, 'energy_loss_in_pipes'] = energy_loss_in_pipes

# Calculate heating costs
devices_data['central_heating_cost'] = devices_data['energy_consumption_in_kJ_per_hour'] + devices_data['energy_loss_in_pipes']
devices_data['local_heating_cost'] = devices_data['energy_consumption_in_kJ_per_hour'] * 1.25

# Assign heating methods based on cost
devices_data['heating_method'] = np.where(
    devices_data['central_heating_cost'] < devices_data['local_heating_cost'],
    'central',
    'local'
)

# Create graph for pipe connections
G = nx.Graph()
for _, row in pipe_connections_data.iterrows():
    node_a = row['node_a']
    node_b = row['node_b']
    loss_of_energy = row['loss_of_energy']
    G.add_edge(node_a, node_b, weight=loss_of_energy)

# Find paths for central heating devices
central_heating_devices = devices_data[devices_data['heating_method'] == 'central']

def find_minimum_paths(graph, source_node, target_nodes):
    min_paths = {}
    for target in target_nodes:
        try:
            # Find the shortest path by weight
            path = nx.shortest_path(graph, source=source_node, target=target, weight='weight')
            path_cost = sum(graph[u][v]['weight'] for u, v in zip(path[:-1], path[1:]))
            min_paths[target] = {'path': path, 'cost': path_cost}
        except nx.NetworkXNoPath:
            print(f"No path found from {source_node} to {target}.")
            min_paths[target] = {'path': None, 'cost': float('inf')}
    return min_paths

# Map devices to graph nodes and find minimum paths
source_node = 'boiler'
central_device_nodes = central_heating_devices['nearest_pipe_node'].tolist()
min_paths = find_minimum_paths(G, source_node, central_device_nodes)

# Calculate total costs
total_central_heating_cost = central_heating_devices['central_heating_cost'].sum()
total_local_heating_cost = devices_data[devices_data['heating_method'] == 'local']['local_heating_cost'].sum()

# Output results
print("Total Central Heating Cost:", total_central_heating_cost)
print("Total Local Heating Cost:", total_local_heating_cost)
print("Optimal Paths for Central Heating Devices:")

for target, details in min_paths.items():
    if details['path']:
        print(f"Device Node: {target}, Path: {details['path']}, Cost: {details['cost']}")
    else:
        print(f"Device Node: {target}, No valid path found.")


Total Central Heating Cost: 6462.309786335738
Total Local Heating Cost: 575.0
Optimal Paths for Central Heating Devices:
Device Node: shower_a, Path: ['boiler', 'n1', 'd1', 'd2', 'd3', 'a1', 'shower_a'], Cost: 12
Device Node: bathtub_a, Path: ['boiler', 'n1', 'd6', 'd7', 'd8', 'a2', 'bathtub_a'], Cost: 12
Device Node: bath_sink_a, Path: ['boiler', 'n1', 'd6', 'd7', 'd8', 'a2', 'a3', 'bath_sink_a'], Cost: 14
Device Node: kitchen_sink_a, Path: ['boiler', 'n1', 'd1', 'd2', 'd3', 'a6', 'a5', 'kitchen_sink_a'], Cost: 15
Device Node: shower_b, Path: ['boiler', 'n1', 'd6', 'd7', 'd8', 'd9', 'b3', 'b1', 'shower_b'], Cost: 16
Device Node: bathtub_b, Path: ['boiler', 'n1', 'd6', 'd7', 'd8', 'd9', 'b2', 'bathtub_b'], Cost: 14
Device Node: bath_sink_b, Path: ['boiler', 'n1', 'd6', 'd7', 'd8', 'd9', 'b3', 'bath_sink_b'], Cost: 13
Device Node: loo_sink_b, Path: ['boiler', 'n1', 'd6', 'd7', 'd8', 'd9', 'b3', 'b4', 'loo_sink_b'], Cost: 15
Device Node: kitchen_sink_b, Path: ['boiler', 'n1', 'd1', 'd2',